In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tcn import TCN
from kerastuner.tuners import RandomSearch

# ------------------ Utility Functions ------------------ #
# Load and preprocess the dataset
def load_and_preprocess_data(filepath):
    data = pd.read_csv(filepath)
    data['date'] = pd.to_datetime(data['date'])
    df_grouped = data.groupby(['date', 'generation_type'], as_index=False)['megawatt_hours'].sum()
    return df_grouped

# Ensure all generation types are represented for every date
def standardize_generation_types(df, unique_types):
    all_dates = df['date'].unique()
    standardized_rows = [
        {'date': date, 'generation_type': g_type, 'megawatt_hours': 0}
        for date in all_dates
        for g_type in unique_types
        if g_type not in df[df['date'] == date]['generation_type'].values
    ]
    return pd.concat([df, pd.DataFrame(standardized_rows)], ignore_index=True)

# Scale the data
def scale_data(df):
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    return scaled_data, scaler

# Create input-output sequences for time-series data
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i + n_steps])
        y.append(data[i + n_steps])
    return np.array(X), np.array(y)

# Forecast future energy generation
def forecast_future(model, data_scaled, scaler, n_steps, forecast_horizon, columns):
    forecast_input = data_scaled[-n_steps:].reshape(1, n_steps, data_scaled.shape[1])
    forecast = []
    for _ in range(len(forecast_horizon)):
        prediction = model.predict(forecast_input)
        forecast.append(prediction[0])
        forecast_input = np.append(forecast_input[:, 1:, :], prediction.reshape(1, 1, -1), axis=1)
    forecast_rescaled = scaler.inverse_transform(forecast)
    return pd.DataFrame(forecast_rescaled, index=forecast_horizon, columns=columns)

# Plot forecasted values
def plot_forecasts(df_pivot, forecast_df, title):
    combined = pd.concat([df_pivot, forecast_df])
    plt.figure(figsize=(12, 6))
    for column in df_pivot.columns:
        plt.plot(combined.index, combined[column], label=column)
    plt.title(title)
    plt.xlabel("Date")
    plt.ylabel("Energy Generation (MWh)")
    plt.legend(loc='upper left', bbox_to_anchor=(1.05, 1))
    plt.tight_layout()
    plt.show()

# ------------------ Model Functions ------------------ #
# Build the TCN model for Keras Tuner
def build_tcn_model(hp):
    dilation_depth = hp.Int('dilation_depth', min_value=2, max_value=6, step=2)
    dilations = [2**i for i in range(dilation_depth)]
    
    model = Sequential([
        Input(shape=(n_steps, X_train.shape[2])),
        TCN(
            nb_filters=hp.Int('nb_filters', min_value=32, max_value=128, step=16),
            kernel_size=hp.Choice('kernel_size', values=[2, 3, 5]),
            nb_stacks=hp.Int('nb_stacks', min_value=1, max_value=2),
            dilations=dilations,
            padding='causal',
            use_skip_connections=hp.Boolean('use_skip_connections'),
            dropout_rate=hp.Float('dropout_rate', min_value=0.0, max_value=0.5, step=0.1),
            return_sequences=False,
            activation=hp.Choice('activation', values=['relu', 'tanh']),
        ),
        Dense(X_train.shape[2])
    ])
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'rmsprop']),
        loss='mse',
        metrics=['mae']
    )
    return model

# ------------------ Main Script ------------------ #
if __name__ == "__main__":
    filepath = r'data\processed\canada_energy.csv'

    # Load and process data
    df_grouped = load_and_preprocess_data(filepath)
    unique_generation_types = df_grouped['generation_type'].unique()
    df_standardized = standardize_generation_types(df_grouped, unique_generation_types)
    df_pivot = df_standardized.pivot(index='date', columns='generation_type', values='megawatt_hours').fillna(0)
    df_pivot[df_pivot < 0] = 0

    # Scale data
    data_scaled, scaler = scale_data(df_pivot)

    # Define time steps
    n_steps = 36
    X, y = create_sequences(data_scaled, n_steps)
    test_size = 24
    train_data, test_data = data_scaled[:-test_size], data_scaled[-test_size - n_steps:]
    X_train, y_train = create_sequences(train_data, n_steps)
    X_test, y_test = create_sequences(test_data, n_steps)

    # Hyperparameter tuning with Keras Tuner
    tuner = RandomSearch(
        build_tcn_model,
        objective='val_loss',
        max_trials=20,
        executions_per_trial=2,
        directory='tuner_results',
        project_name='tcn_energy_forecasting'
    )
    tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32, verbose=2)

    # Get best hyperparameters and train model
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)
    history = best_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), verbose=2)

    # Forecast future values
    forecast_horizon = pd.date_range(df_pivot.index[-1] + pd.DateOffset(months=1), '2030-12-01', freq='MS')
    forecast_df_tcn = forecast_future(best_model, data_scaled, scaler, n_steps, forecast_horizon, df_pivot.columns)

    # Plot results
    plot_forecasts(df_pivot, forecast_df_tcn, "Energy Generation Forecast to 2030 (TCN)")


TypeError: A `Choice` can contain only `int`, `float`, `str`, or `bool`, found values: [(1, 2, 4, 8), (1, 2, 4, 8, 16, 32), (1, 2, 4, 8, 16, 32, 64)]with types: <class 'tuple'>